In [54]:
import pandas as pd
import numpy as np


In [55]:
file= "Resources/purchase_data.csv"
df = pd.read_csv(file)
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [56]:
num_players = len(df.SN.unique())
total_players_table = pd.DataFrame({"Total Players": [num_players]})
total_players_table


,Total Players
0,576


In [57]:
unique_items = len(df["Item ID"].unique())
df["Price"] = df["Price"].astype(float)
avgprice= round(df.Price.mean(),2)
numpurchase= len(df["Purchase ID"].unique())
total_rev= df["Price"].sum()

In [58]:
purchase_analysis_table= pd.DataFrame({"Number of Unique Items": [unique_items], "Average Price":"$"+str(avgprice),
                                       "Number of Purchases":[numpurchase], "Total Revenue":"$"+str(total_rev)})
purchase_analysis_table

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


In [59]:
genderdf= df.drop_duplicates(subset="SN",keep="first")
group= genderdf.groupby(["Gender"])
gender_count= pd.DataFrame(group.SN.value_counts())
#total and unique
gender_total= genderdf.Gender.count()
gender_unique_count= group["Gender"].count()
#percent of gender count
gender_pct = pd.DataFrame(round((gender_unique_count/gender_total)*100,2))
gender_pct= gender_pct.rename(columns={"Gender":"Percent of Players"})
#count of Gender Count
gender_pctc = pd.DataFrame(gender_total -(gender_total - gender_unique_count))
gender_pctc= gender_pctc.rename(columns={"Gender":"Total Count"})
#combined DF
gender_pct= gender_pct.merge(gender_pctc, on= "Gender", how ="outer")
gender_pctorg = gender_pct[["Total Count","Percent of Players"]]
gender_pctorg

,Total Count,Percent of Players
Gender,,
Female,81,14.06
Male,484,84.03
Other / Non-Disclosed,11,1.91


In [60]:
df["Purchase ID"]= df["Purchase ID"].astype("float")
PAGdf= df.groupby(["Gender"])
PAGdf_count= pd.DataFrame(PAGdf["Purchase ID"].count())
PAGdf_revdf= pd.DataFrame(PAGdf["Price"].sum())
PAG_table= PAGdf_count.merge(PAGdf_revdf, on= "Gender", how= "outer")

PAG_table["Average Purchase Price"]= round((PAG_table["Price"] / PAG_table["Purchase ID"]),2)
PAG_table2= PAG_table.merge(gender_pctc, on= "Gender", how = "outer")
PAG_table2["Avg Total Purchase per Person"]= round((PAG_table2["Price"] / PAG_table2["Total Count"]),2)
PAG_rename = PAG_table2.rename(columns={"Purchase ID":"Purchase Count", "Price":"Total Purchase Value"})
PAG_tablefinal= PAG_rename[["Purchase Count", "Average Purchase Price", 
                            "Total Purchase Value", "Avg Total Purchase per Person"]]
PAG_tablefinal["Average Purchase Price"]=PAG_tablefinal["Average Purchase Price"].map("${:,.2f}".format)
PAG_tablefinal["Total Purchase Value"]=PAG_tablefinal["Total Purchase Value"].map("${:,.2f}".format)
PAG_tablefinal["Avg Total Purchase per Person"]=PAG_tablefinal["Avg Total Purchase per Person"].map("${:,.2f}".format)
PAG_tablefinal

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [89]:
#create bins
bins = [0,9,14,19,24,29,34,39,100]
#create labels
labels= ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
#add bin column to DF
df["Age Group"]= pd.cut(df["Age"], bins, labels= labels)
#drop duplicates in SN
df2= df.drop_duplicates(subset="SN",keep="first")
#groupby Age Group
agedf= df2.groupby(["Age Group"])
#get total and group count
age_total= df2["Age"].count()
age_group_total= agedf["Age"].count()
#df for count
age_count= pd.DataFrame(age_total-(age_total - age_group_total))
#df for percent
age_pct = pd.DataFrame(round((age_group_total/age_total)*100,2))
age_table= age_count.merge(age_pct, on="Age Group", how="outer")
#rename columns
age_final = age_table.rename(columns={"Age_x":"Total Count", "Age_y":"Percentage of Players"})

df["Price"]= df["Price"].astype("float")
agerevdf= df.groupby(["Age Group"])
agerev_count= pd.DataFrame(agerevdf["Purchase ID"].count())
agerevdf= pd.DataFrame(agerevdf["Price"].sum())
agerev_table= agerev_count.merge(agerevdf, on= "Age Group", how= "outer")
age_final2= age_final.merge(agerev_table, on="Age Group", how="outer")
age_final2["Avg Total Purchase per Person"]= round((age_final2["Price"] / age_final2["Total Count"]),2)
age_final2["Avg Purchase Price"]= round((age_final2["Price"] / age_final2["Purchase ID"]),2)
age_finalrename= age_final2.rename(columns={"Purchase ID":"Purchase Count", "Price": "Total Purchase Value"})
agefinal= age_finalrename[["Purchase Count", "Avg Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"]]
#format column values
agefinal["Avg Purchase Price"]=agefinal["Avg Purchase Price"].map("${:,.2f}".format)
agefinal["Total Purchase Value"]=agefinal["Total Purchase Value"].map("${:,.2f}".format)
agefinal["Avg Total Purchase per Person"]=agefinal["Avg Total Purchase per Person"].map("${:,.2f}".format)
agefinal

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

,Purchase Count,Avg Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [105]:
sndf = df.groupby(["SN"])
sndfpc = sndf["Purchase ID"].count()
sndfprice= sndf["Price"].sum()
sndf1= pd.DataFrame({"Purchase Count": sndfpc, "Total Purchase Value": sndfprice})
sndf1["Average Purchase Price"]= round((sndfprice / sndfpc),2)
sndfsort= sndf1.sort_values("Total Purchase Value", ascending= False)
sndfsort["Average Purchase Price"]=sndfsort["Average Purchase Price"].map("${:,.2f}".format)
sndfsort["Total Purchase Value"]=sndfsort["Total Purchase Value"].map("${:,.2f}".format)
sndfsort.head(5)

,Purchase Count,Total Purchase Value,Average Purchase Price
SN,,,
Lisosia93,5,$18.96,$3.79
Idastidru52,4,$15.45,$3.86
Chamjask73,3,$13.83,$4.61
Iral74,4,$13.62,$3.40
Iskadarya95,3,$13.10,$4.37


In [109]:
mpidf= df.groupby(["Item ID", "Item Name"])
mpidfpc = mpidf["Purchase ID"].count()
mpidfprice= mpidf["Price"].sum()
mpidf1= pd.DataFrame({"Purchase Count": mpidfpc, "Total Purchase Value": mpidfprice})
mpidf1["Item Price"]= round((mpidfprice / mpidfpc),2)
mpidfsort= mpidf1.sort_values("Purchase Count", ascending= False)
mpidfsort["Item Price"]=mpidfsort["Item Price"].map("${:,.2f}".format)
mpidfsort["Total Purchase Value"]=mpidfsort["Total Purchase Value"].map("${:,.2f}".format)
mpidfsort.head()

,,Purchase Count,Total Purchase Value,Item Price
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76,$4.23
145,Fiery Glass Crusader,9,$41.22,$4.58
108,"Extraction, Quickblade Of Trembling Hands",9,$31.77,$3.53
82,Nirvana,9,$44.10,$4.90
19,"Pursuit, Cudgel of Necromancy",8,$8.16,$1.02


In [110]:
mpidfsort2= mpidf1.sort_values("Total Purchase Value", ascending= False)
mpidfsort2["Item Price"]=mpidfsort2["Item Price"].map("${:,.2f}".format)
mpidfsort2["Total Purchase Value"]=mpidfsort2["Total Purchase Value"].map("${:,.2f}".format)
mpidfsort2.head()

,,Purchase Count,Total Purchase Value,Item Price
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76,$4.23
82,Nirvana,9,$44.10,$4.90
145,Fiery Glass Crusader,9,$41.22,$4.58
92,Final Critic,8,$39.04,$4.88
103,Singed Scalpel,8,$34.80,$4.35


In [ ]:
# Observances:

# 1) Male players comprise 84% of the sales. 
    #However, without including the  number of players who did not make a purhcase, we cannot determine whether male players
    #are more prone to make purchases.
# 2) Female players included in dataset spend 10% more than their male counterparts. 